# 1.) Launch and Connect to the Simulation
Import useful python packages

In [ ]:
import carla
import random
from carla import WeatherParameters

Connect to the client

In [ ]:
# Start localhost world
client = carla.Client('localhost', 2000)

# 2.) Define the Driving Environment
First we need to choose a town to drive in. In this sitation I wanted my world to be unique so I used Town 1

In [ ]:
# Load world
world = client.load_world('Town01')

# Load Vehicle
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
# Get Spawn Points
spawn_points = world.get_map().get_spawn_points()

# Get pedestrian blueprints
pedestrian_blueprints = world.get_blueprint_library().filter('walker.pedestrian.*')

# Define a function to spawn a pedestrian
def spawn_pedestrian(world, pedestrian_blueprints, spawn_point):
    pedestrian_bp = random.choice(pedestrian_blueprints)
    pedestrian = world.try_spawn_actor(pedestrian_bp, spawn_point)
    return pedestrian

# Spawn pedestrians
num_pedestrians = 50

for _ in range(num_pedestrians):
    spawn_point = random.choice(spawn_points)
    pedestrian = spawn_pedestrian(world, pedestrian_blueprints, spawn_point)


Spawn 50 vehicles randomly distributed throughout the map, for each spawn point, we choose a random vehicle from the blueprint library

In [ ]:
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints),
random.choice(spawn_points))

# 3.) Create and Instrument Our Vehicle
Create Our First Ego Vehicle

In [ ]:
ego_vehicle = world.spawn_actor(random.choice(vehicle_blueprints),
random.choice(spawn_points))

Move the spectatore view to look at the right of our ego vehicle.

In [ ]:
spectator = world.get_spectator()
transform = ego_vehicle.get_transform()
spectator.set_transform(carla.Transform(transform.location +
carla.Location(z=50),
carla.Rotation(pitch=-90)))

Create a transform to place the camera on top of the vehicle

In [ ]:
camera_init_trans = carla.Transform(carla.Location(z=1.5))

We create the camera through a blueprint that defines its properties

In [ ]:
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

We spawn the camera and attach it to our ego vehicle

In [ ]:
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

Create a transform to place a GPS on top of the vehicle

In [ ]:
# Create a GPS sensor blueprint
gps_bp = world.get_blueprint_library().find('sensor.other.gnss')

# Set GPS sensor attributes, if needed (e.g. noise, frequency)
# gps_bp.set_attribute('noise_stddev', '1')

# Create a transform to place the GPS sensor on top of the vehicle
gps_init_trans = carla.Transform(carla.Location(z=1.5))

# Spawn the GPS sensor and attach it to the vehicle
gps_sensor = world.spawn_actor(gps_bp, gps_init_trans, attach_to=ego_vehicle)


Create a transform to place a Lidar on top of the vehicle

In [ ]:
# Create a LiDAR sensor blueprint
lidar_bp = world.get_blueprint_library().find('sensor.lidar.ray_cast')

# Set LiDAR sensor attributes (e.g. channels, range, points_per_second, etc.)
lidar_bp.set_attribute('channels', '32')
lidar_bp.set_attribute('range', '100')
lidar_bp.set_attribute('points_per_second', '100000')

# Create a transform to place the LiDAR sensor on top of the vehicle
lidar_init_trans = carla.Transform(carla.Location(z=1.5))

# Spawn the LiDAR sensor and attach it to the vehicle
lidar_sensor = world.spawn_actor(lidar_bp, lidar_init_trans, attach_to=ego_vehicle)



# 4.) Run the Simulation and Record Data

Set all the vehicles in motion and right away start recording camera, gps, and lidar data with PyGameCallback. This saves images into the "out" folder until the simulation has stopped

In [ ]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)
# Start camera with PyGame callback
camera.listen(lambda image: image.save_to_disk('nihal/out/%06d.png' % image.frame))

# Define a callback to process the GPS data
def process_gps_data(gnss_data):
    with open("nihal/out/gps_data.txt", "a") as f:
        f.write("GNSS data: Lat={}, Lon={}, Alt={}\n".format(gnss_data.latitude, gnss_data.longitude, gnss_data.altitude))

# Listen for GPS data
gps_sensor.listen(process_gps_data)

# Define a callback to process the LiDAR data
def process_lidar_data(lidar_data):
    lidar_data.save_to_disk('nihal/out/Lidar/%06d.ply' % lidar_data.frame)

# Listen for LiDAR data
lidar_sensor.listen(process_lidar_data)